In [4]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import concurrent.futures

src_dir = Path(
    "/workspaces/cv/data/surveys/action-aviation-multicamera/20250331_west_coast_4camera_test/flight_1/cameras"
)
nh = 3
nw = 3
odir = Path(f"/workspaces/cv/.tmp/20250331_west_coast_4camera_test_flight_1_crop_{nh}x{nw}")
quality = 95


def do(imgpath):
    img = Image.open(imgpath)
    # Get crops:
    for yi in range(nh):
        for xi in range(nw):
            x0 = int(img.width // nw * xi)
            y0 = int(img.height // nh * yi)
            x1 = int(img.width // nw * (xi + 1))
            y1 = int(img.height // nh * (yi + 1))
            this_odir = odir / imgpath.parent.relative_to(src_dir)
            opath = this_odir / f"{imgpath.stem}_{x0}_{y0}.jpg"
            if opath.exists():
                continue
            crop = img.crop((x0, y0, x1, y1))
            this_odir.mkdir(parents=True, exist_ok=True)
            tmp_path = opath.with_suffix(".tmp.jpg")
            crop.save(tmp_path, quality=quality)
            tmp_path.rename(opath)


imgpaths = list(sorted(src_dir.rglob("*.JPG")))
# imgpaths = [i for i in imgpaths if i.name == "_09R9718.JPG"]

# Create all the thumbs:
pbar = tqdm(total=len(imgpaths), desc="Processing images")
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(do, i) for i in imgpaths]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"Error processing thumbnail: {e}")
            continue
        pbar.update(1)
pbar.close()


Processing images: 100%|██████████| 11844/11844 [02:16<00:00, 86.88it/s] 
